In [40]:
import numpy as np
import pandas as pd
import datetime
import pandoc

In [2]:
assessment = pd.read_csv('Historic_Secured_Property_Tax_Rolls.csv',parse_dates=True)

C:\Users\Nishu\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
assessment.columns = ['Closed_Roll_Fiscal_Year',
'Property_Location',
'Neighborhood_Code',
'Neighborhood_Code_Definition',
'Block_and_Lot_Number',
'Volume_Number',
'Property_Class_Code',
'Property_Class_Code_Definition',
'Year_Property_Built',
'Number_of_Bathrooms',
'Number_of_Bedrooms',
'Number_of_Rooms',
'Number_of_Stories',
'Number_of_Units',
'Characteristics_Change_Date',
'Zoning_Code',
'Construction_Type',
'Lot_Depth',
'Lot_Frontage',
'Property_Area_in_Square_Feet',
'Basement_Area',
'Lot_Area',
'Lot_Code',
'Prior_Sales_Date',
'Recordation_Date',
'Document_Number',
'Document_Number_2',
'Tax_Rate_Area_Code',
'Percent_of_Ownership',
'Closed_Roll_Exemption_Type_Code',
'Closed_Roll_Exemption_Type_Code_Definition',
'Closed_Roll_Status_Code',
'Closed_Roll_Misc_Exemption_Value',
'Closed_Roll_Homeowner_Exemption_Value',
'Current_Sales_Date',
'Closed_Roll_Assessed_Fixtures_Value',
'Closed_Roll_Assessed_Improvement_Value',
'Closed_Roll_Assessed_Land_Value',
'Closed_Roll_Assessed_Personal_Prop_Value',
'Zipcode_of_Parcel',
'Supervisor_District',
'Neighborhoods_-_Analysis_Boundaries',
'Location']

### Problem-1

In [7]:
## Most common class 
a,b = np.unique(assessment.Property_Class_Code,return_counts=True)
c = zip(a,b)
print("D class code has most no of assessments")

D class code has most no of assessments


C:\Users\Nishu\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [8]:
print("No of assessments in class D : ",len(assessment[assessment.Property_Class_Code == 'D']))

('No of assessments in class D : ', 758861)


In [9]:
print("No of total assessments :",len(assessment))

('No of total assessments :', 1612110)


In [10]:
## Question-1 Fraction of assessments for most popular class 'D' is:

print("The Frcation of assessments for most popular class is :",758861./1612110)

('The Frcation of assessments for most popular class is :', 0.4707253227137106)


### Problem-2

In [11]:
## Function returning median assessment improvement value of latest assessment year for non-zero assessments
def median_assessments_improvement(assessment):
    assess_improvement=assessment[assessment['Closed_Roll_Assessed_Improvement_Value'] != 0]
    assess_improvement['Closed_Roll_Fiscal_Year_max']=assess_improvement.groupby('Block_and_Lot_Number')['Closed_Roll_Fiscal_Year'].transform(max)
    final_df = assess_improvement[np.equal(assess_improvement['Closed_Roll_Fiscal_Year'],assess_improvement['Closed_Roll_Fiscal_Year_max'])]
    return(final_df['Closed_Roll_Assessed_Improvement_Value'].median())

In [12]:
print("Median assessment improvement value for latest financial year :",median_assessments_improvement(assessment))

C:\Users\Nishu\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Median assessment improvement value for latest financial year :', 209161.0)


### Problem-3

In [13]:
def average_improvement_each_neighborhood(assessment):
    average_improvement = []
    assess_improvement=assessment[assessment['Closed_Roll_Assessed_Improvement_Value'] != 0]
    average_improvement = assess_improvement.groupby('Neighborhood_Code')['Closed_Roll_Assessed_Improvement_Value'].mean()
    return(average_improvement.max() - average_improvement.min())

In [14]:
avg_diff = average_improvement_each_neighborhood(assessment)

In [15]:
print("The average difference between greatest & least average assess improvement value of each non-zero neighborhood code :",average_improvement_each_neighborhood(assessment))

('The average difference between greatest & least average assess improvement value of each non-zero neighborhood code :', 5073477.6646721791)


### Problem-4

In [16]:
def rate_of_land_values(assessment):
    assess_value = assessment[assessment['Closed_Roll_Assessed_Land_Value'] != 0.0]
    min_land_value = min(assess_value.groupby('Closed_Roll_Fiscal_Year')['Closed_Roll_Assessed_Land_Value'].max())
    max_land_value = max(assess_value.groupby('Closed_Roll_Fiscal_Year')['Closed_Roll_Assessed_Land_Value'].max())
    
    ## Used the formula P = P0*e^(rate*time_period)
    ## P = max_land_value
    ## P0 = min_land_value
    ## Time-period = 6 years
    ## rate = r to be determined
    
    r = np.log(max_land_value/min_land_value)/6
    
    ## The rate of land_value growth over 6 years is:
    print("The rate of land_value growth over 6 years ",r)
    
    ## We are interested in yearly growth, so
    print("The yearly growth rate of land_value :",r/6)

In [17]:
rate_of_land_values(assessment)

('The rate of land_value growth over 6 years ', 0.014427005905268119)
('The yearly growth rate of land_value :', 0.002404500984211353)


### Problem-5

In [4]:
location_non_zero = assessment.loc[(~assessment.Location.isnull())]

In [10]:
location_non_zero.Location = [tuple(each[1:-1].split(','))  for each in location_non_zero.Location]

In [9]:
location_non_zero['latitude'] = [location_non_zero.Location[i][0] for i in location_non_zero.index.values]

In [8]:
location_non_zero['longitude'] = [location_non_zero.Location[i][1] for i in location_non_zero.index.values]

In [16]:
location_non_zero.latitude = location_non_zero.latitude.astype(np.float32)
location_non_zero.longitude = location_non_zero.longitude.astype(np.float32)

In [20]:
list = []
for code in location_non_zero.Neighborhood_Code.unique():
    df = location_non_zero.loc[(location_non_zero.Neighborhood_Code) == code]
    area = np.std(df.latitude)*np.std(df.longitude)*np.pi
    list.append([code,area])

In [28]:
area = [each[1] for each in list]

In [35]:
def area_by_neighborhood(assessment):
    location_non_zero = assessment.loc[(~assessment.Location.isnull())]
    location_non_zero.Location = [tuple(each[1:-1].split(','))  for each in location_non_zero.Location]
    location_non_zero['latitude'] = [location_non_zero.Location[i][0] for i in location_non_zero.index.values]
    location_non_zero['longitude'] = [location_non_zero.Location[i][1] for i in location_non_zero.index.values]
    location_non_zero.latitude = location_non_zero.latitude.astype(np.float32)
    location_non_zero.longitude = location_non_zero.longitude.astype(np.float32)
    list = []
    for code in location_non_zero.Neighborhood_Code.unique():
        df = location_non_zero.loc[(location_non_zero.Neighborhood_Code) == code]
        area = np.std(df.latitude)*np.std(df.longitude)*np.pi
        list.append([code,area])
        ar = [each[1] for each in list]
    return(max(ar))

In [38]:
print("Area of larget neighborhood using ellipse ",area_by_neighborhood(assessment))

C:\Users\Nishu\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Nishu\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Area of larget neighborhood using ellipse ', 0.0019197269946201632)


### Problem-6

In [18]:
def mean_units_diff(assessment):
    assessment['Year_Property_Built'].fillna(value=0.0,inplace=True)
    non_zero_units = assessment[assessment['Number_of_Units']!= 0]
    non_zero_unit_geq_1950=non_zero_units[non_zero_units['Year_Property_Built'] >= 1950.0]
    non_zero_unit_geq_1950['Closed_Roll_Fiscal_Year_min'] = non_zero_unit_geq_1950.groupby('Block_and_Lot_Number')['Closed_Roll_Fiscal_Year'].transform(min)
    non_zero_unit_lt_1950=non_zero_units[non_zero_units['Year_Property_Built'] < 1950.0]
    non_zero_unit_lt_1950['Closed_Roll_Fiscal_Year_min'] = non_zero_unit_lt_1950.groupby('Block_and_Lot_Number')['Closed_Roll_Fiscal_Year'].transform(min)
    final_geq_1950 = non_zero_unit_geq_1950[np.equal(non_zero_unit_geq_1950['Closed_Roll_Fiscal_Year_min'],non_zero_unit_geq_1950['Closed_Roll_Fiscal_Year_min'])]
    final_lt_1950 = non_zero_unit_lt_1950[np.equal(non_zero_unit_lt_1950['Closed_Roll_Fiscal_Year_min'],non_zero_unit_lt_1950['Closed_Roll_Fiscal_Year_min'])]
    return(np.subtract(final_geq_1950['Number_of_Units'].mean(),final_lt_1950['Number_of_Units'].mean()))

In [19]:
print("Difference between average no of units build after 1950 and before 1950 :",mean_units_diff(assessment))

C:\Users\Nishu\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Nishu\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Difference between average no of units build after 1950 and before 1950 :', 0.5828151741183123)


### Problem-7

In [49]:
def ratio_means_by_zipcode(assessment):
    non_zero = assessment.loc[(assessment.Number_of_Bedrooms !=0)&(assessment.Number_of_Units != 0)]
    avg_no_bedrooms = non_zero.groupby('Zipcode_of_Parcel').Number_of_Bedrooms.mean()
    avg_no_units = non_zero.groupby('Zipcode_of_Parcel').Number_of_Units.mean()
    ratio_of_means = np.divide(avg_no_bedrooms.values,avg_no_units.values)
    return(ratio_of_means.max())

In [50]:
print("Maximum average number of bedrooms per unit by zipcode :",ratio_means_by_zipcode(assessment))

('Maximum average number of bedrooms per unit by zipcode :', 3.8768601648013776)


### Problem-8

In [52]:
property_area_by_zipcode = assessment.groupby('Zipcode_of_Parcel').Property_Area_in_Square_Feet.sum()

In [53]:
lot_area_by_zipcode = assessment.groupby('Zipcode_of_Parcel').Lot_Area.sum()

In [54]:
ratio_property_to_Lot_by_zip = np.divide(property_area_by_zipcode.values,lot_area_by_zipcode.values)

In [56]:
ratio_property_to_Lot_by_zip.max()

12.486006304191623

In [57]:
def property_to_lot_by_zip(assessment):
    property_area_by_zipcode = assessment.groupby('Zipcode_of_Parcel').Property_Area_in_Square_Feet.sum()
    lot_area_by_zipcode = assessment.groupby('Zipcode_of_Parcel').Lot_Area.sum()
    ratio_property_to_Lot_by_zip = np.divide(property_area_by_zipcode.values,lot_area_by_zipcode.values)
    return(ratio_property_to_Lot_by_zip.max())

In [59]:
print("The ratio of maximum property to surface are by zipcode :",property_to_lot_by_zip(assessment))

('The ratio of maximum property to surface are by zipcode :', 12.486006304191623)
